# Geostructures: Shape Collections

Geostructures provides two objects for working with collections of shapes:
* `FeatureCollection`: A collection of shapes with no added requirements
* `Track`: A chronologically-ordered collection of shapes (requires that each shape be time-bounded)

This notebook will demonstrate:
* Creating and plotting shape collections
* Operations with shape collections
* Converting collections to alternate formats (GeoJSON, GeoPandas, ESRI Shapefile)
* Track Operations

In [ ]:
# If you've git cloned this repo, run this cell to add `geostructures` to your python path
import sys
sys.path.insert(0, '../')

## FeatureCollections

FeatureCollections are just that - collections of features. Simply pass a list of shapes to create one.

In [ ]:
from datetime import datetime

from geostructures import *
from geostructures.collections import FeatureCollection, Track
from geostructures.visualization.plotly import draw_collection

# To create a FeatureCollection, simply pass it a list of geoshapes
collection = FeatureCollection(
    [
        GeoCircle(Coordinate(-0.118092, 51.509865), 500),
        GeoEllipse(Coordinate(-0.118072, 51.502365), 2000, 400, 80),
        GeoCircle(Coordinate(-0.141092, 51.529865), 500),
        GeoRing(Coordinate(-0.116092, 51.519865), inner_radius=800, outer_radius=1000),
        GeoRing(Coordinate(-0.101092, 51.514865), inner_radius=300, outer_radius=500, angle_min=60, angle_max=190),
    ]
)

# Plot using plotly
fig = draw_collection(collection)
fig.update_layout(
    width=1100, 
    height=500, 
    mapbox_zoom=10, 
)

## Filtering by Intersection/Containment

You can filter a collection down to only those shapes which intersect or are contained by another shape of your choosing

In [ ]:
intersecting_circle = GeoCircle(Coordinate(-0.098092, 51.509865), 1500)

filtered_intersection = collection.filter_by_intersection(intersecting_circle)
print(f"Intersection: filtered to {len(filtered_collection)} shapes")

filtered_intersection = collection.filter_by_contains(intersecting_circle)
print(f"Contains: filtered to {len(filtered_collection)} shapes")

fig2 = draw_collection(FeatureCollection([intersecting_circle]), color='blue', fig=fig)
fig2.update_layout(
    width=1100, 
    height=500, 
    mapbox_zoom=10, 
)

Alternatively if you just want to test whether a collection intersects your shape, you can use the `.intersects()` method.

In [ ]:
print("Collection intersects shape?", collection.intersects(intersecting_circle))

### Collections Conversions

FeatureCollections and Tracks come with convenience methods for converting your structures into GeoJSON and GeoPandas DataFrames, or reading either of these into a collection.

Just like other methods, any properties stored on the shapes will be injected into the DataFrame/json.

#### GeoPandas

In [ ]:
# Convert a collection to a GeoDataFrame
df = collection.to_geopandas()
print(df.head(), end='\n\n')

# Read a FeatureCollection from a GeoDataFrame
collection = FeatureCollection.from_geopandas(df)
print(collection)

#### GeoJSON

In [ ]:
# Convert a collection to GeoJSON
gjson = collection.to_geojson()

# Read a FeatureCollection from GeoJSON
collection = FeatureCollection.from_geojson(gjson)

## Tracks

Tracks can do everything that a FeatureCollection can do, but the time requirement for all shapes allows for a number of time-focused analytics.

Just like FeatureCollections, you can create a Track by passing it a list of shapes. Unlike FeatureCollections, though, every 
shape must have an associated time or TimeInterval.

In [ ]:
# In order to create a track, all shapes must be bound by a datetime or timeinterval
track = Track(
    [
        GeoPoint(Coordinate(-0.104154, 51.511920), dt=datetime(2020, 1, 1, 9, 5)),
        GeoPoint(Coordinate(-0.096533, 51.511903), dt=datetime(2020, 1, 1, 9, 23)),
        GeoPoint(Coordinate(-0.083765, 51.514423), dt=datetime(2020, 1, 1, 9, 44)),
        GeoPoint(Coordinate(-0.087478, 51.508595), dt=datetime(2020, 1, 1, 10, 1)),
    ]
)

You can get metrics on the intervals between your declared shapes, such as the average speed required to move from point A to point B (assuming straight line movement, in meters per second)

In [ ]:
print("Distances between points:", track.centroid_distances)
print("Time differences:", track.time_start_diffs)
print("Average speed:", track.speed_diffs)

#### Slicing by Time

In [ ]:
track[datetime(2020, 1, 1, 9, 23):datetime(2020, 1, 1, 9, 45)]